In [1]:
from pyspark.sql import SparkSession
from delta.tables import *
from delta import *

In [2]:
 spark = SparkSession.builder \
        .appName("ProcessJSONToDelta") \
        .config("spark.jars.packages", "io.delta:delta-spark_2.12:3.3.0") \
        .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
        .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
        .getOrCreate()

25/02/06 15:08:08 WARN Utils: Your hostname, NB-WXET1590 resolves to a loopback address: 127.0.1.1; using 172.30.88.241 instead (on interface eth0)
25/02/06 15:08:08 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/avyas/miniconda3/envs/testairbyte/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/avyas/.ivy2/cache
The jars for the packages stored in: /home/avyas/.ivy2/jars
io.delta#delta-spark_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-1d961a6f-a845-49fd-adda-57ccac3beb70;1.0
	confs: [default]
	found io.delta#delta-spark_2.12;3.3.0 in central
	found io.delta#delta-storage;3.3.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
:: resolution report :: resolve 316ms :: artifacts dl 13ms
	:: modules in use:
	io.delta#delta-spark_2.12;3.3.0 from central in [default]
	io.delta#delta-storage;3.3.0 from central in [default]
	org.antlr#antlr4-runtime;4.9.3 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   |   0  

In [ ]:
data = spark.range(5, 10)
data.write.format("delta").mode("overwrite").save("/tmp/delta-table")


In [ ]:
delta_table = DeltaTable.forPath(spark, '/tmp/delta-table')

In [ ]:
spark.sql ("""CREATE TABLE delta_table
                USING delta
                LOCATION '/tmp/delta-table'""").show()

In [ ]:
spark.sql("""Select * from delta_table """).show()

In [ ]:
# Upsert (merge) new data
new_data = spark.range(0, 25)

new_data.createOrReplaceTempView("new_data")

print (spark.sql("""Select * from new_data """).show())
# Merge new data with Delta table
merge_query = """
    MERGE INTO delta_table
    USING new_data
    ON delta_table.id = new_data.id
    WHEN MATCHED THEN
        UPDATE SET *
    WHEN NOT MATCHED THEN
        INSERT *
"""
spark.sql(merge_query)

In [ ]:
delta_table = DeltaTable.forPath(spark, '/tmp/delta-table')
delta_table.toDF().show(30)